In [ ]:
import os

from trulens.connectors.snowflake import SnowflakeConnector

os.environ["TRULENS_OTEL_TRACING"] = "1"
connection_params = {  # creds for QA6
    "account": "mlplatformtest.qa6.us-west-2.aws",
    "user": "dhuang",
    "password": "nun-DJQ0fea1fun4rqv",
    "database": "DHUANG_TEST",
    "warehouse": "DHUANG",
    "role": "ENGINEER",
    "schema": "E2E",
}

sf_connector = SnowflakeConnector(**connection_params)

In [ ]:
from trulens.apps.app import TruApp
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class TestApp:
    @instrument(span_type=SpanAttributes.SpanType.RECORD_ROOT)
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[1]": args[1],
        }
    )
    def nested2(self, query: str) -> str:
        nested_result = ""

        try:
            nested_result = self.nested3(query)
        except Exception:
            pass

        return f"nested2: {nested_result}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested3_ex": exception.args if exception else None,
            "nested3_ret": ret,
            "selector_name": "special",
            "cows": "moo",
        }
    )
    def nested3(self, query: str) -> str:
        if query == "throw":
            raise ValueError("nested3 exception")
        return "nested3"


app = TestApp()

In [ ]:
TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    object_type="RANDOM_UNSUPPORTED",
)

TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
)

TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    main_method=app.respond_to_query,
)
# these should all throw exceptions

In [ ]:
tru_recorder = TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    connector=sf_connector,
    main_method=app.respond_to_query,
    # object_type default to "EXTERNAL AGENT" when snowflake connector is used
)

print(tru_recorder.snowflake_object_type)  # EXTERNAL AGENT
print(tru_recorder.snowflake_object_name)  # CUSTOM_APP
print(tru_recorder.snowflake_object_version)

version_df = tru_recorder.snowflake_app_dao.list_agent_versions("custom_app")
version_df

In [ ]:
tru_recorder.snowflake_app_dao.list_agents()

In [ ]:
# tru_recorder.delete_snowflake_app()

tru_recorder.snowflake_app_dao.list_agents()

In [ ]:
tru_recorder_v1 = TruApp(
    app,
    app_name="custom_app_multi_ver",
    app_version="v1",
    connector=sf_connector,
    main_method=app.respond_to_query,
)

assert tru_recorder_v1.snowflake_app_dao.check_agent_exists(
    "custom_app_multi_ver"
)

# Create app version 2.
tru_recorder_v2 = TruApp(
    app,
    app_name="custom_app_multi_ver",
    app_version="v2",
    connector=sf_connector,
    main_method=app.respond_to_query,
)

assert tru_recorder_v2.snowflake_app_dao is not None

assert tru_recorder_v2.snowflake_app_dao.check_agent_exists(
    "custom_app_multi_ver"
)

versions_df_1 = tru_recorder_v1.snowflake_app_dao.list_agent_versions(
    "custom_app_multi_ver"
)

versions_df_2 = tru_recorder_v2.snowflake_app_dao.list_agent_versions(
    "custom_app_multi_ver"
)

## Add runs to agent

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

run_config = RunConfig(
    run_name="test_run_xyz",
    description="desc",
    dataset_name="TEST_TABEL",
    label="label",
    dataset_col_spec={"col1": "int", "col2": "string"},
)  # type: ignore
new_run: Run = tru_recorder.add_run(run_config=run_config)

run: Run = tru_recorder.get_run("test_run_10")

# so forth

print(run)
print(new_run)

In [ ]:
run.run_name

In [ ]:
run_config_2 = RunConfig(
    run_name="test_run_2",
    description="desc_2",
    dataset_name="db.schema.table",
    dataset_col_spec={"col1": "int", "col2": "string"},
)
run_2 = tru_recorder.add_run(run_config=run_config_2)

runs = tru_recorder.list_runs()

In [ ]:
# run_2.delete()
runs = tru_recorder.list_runs()

In [ ]:
runs